## DATA CLEAN

In [26]:
import re
import pandas as pd
import numpy as np
import datetime

In [110]:
'''importation du df'''
df = pd.read_csv(r'C:\Users\Qw\Desktop\projet_gaming\vgsales_complet.csv')



'''comparer les nom des jeux entre fichier de laurent et vg_sales (metacritic vs vg_sales)'''
samegame=[]
for i, j in zip(df['Name'],df['gamename']):
    if i in j or j in i:
        samegame.append(1)
    else:
        samegame.append(0)
df['same_game']=samegame 




'''recupération des données : '''
    # extraire les données de la colonne userscorewrap pour corriger les erreurs de
    #la colonne ratings et nb_votes
y=[]
for i in df['userscorewrap']:
    x = re.findall(r'[0-9\.0-9]+',i)
    y.append(x)
df['userscorewrap2']=y
    # remplacer les listes vides ou avec une seul valeur par [0,0] afin 
    # d'éviter l'erreur de "length..."
for i in df['userscorewrap2']:
    if i==[]:
        i.append(0)
        i.append(0)
    if len(i)==1:
        i.clear()
        i.append(0)
        i.append(0)
    # extraire les données de la colonne metascorewrap pour corriger les erreurs de
    #la colonne metascore et nb_critic
n=[]
for i in df['metascorewrap']:
    x = re.findall(r'[0-9\.0-9]+',i)
    n.append(x)
df['metascorewrap2']=n
    # remplacer les listes vides ou avec une seul valeur par [0,0] afin d'éviter l'erreur de "length..."
for i in df['metascorewrap2']:
    if i==[]:
        i.append(0)
        i.append(0)
    if len(i)==1:
        i.clear()
        i.append(0)
        i.append(0)
    # recuperer les données de la colonnes metascorewrap2 et userscorewrap2
x=[]
y=[]
for i in df['userscorewrap2']:
    x.append(i[0])
    y.append(i[1])
df['ratings']=x
df['nb_votes']=y

x=[]
y=[]
for i in df['metascorewrap2']:
    x.append(i[0])
    y.append(i[1])
df['metascore']=x
df['nb_critic']=y

    # recuperer série (si série donc =1 sinon 0)
serie=[]
def extract_series(x): 
    r = re.compile(r"Series:.*\n") 
    result=r.findall(x)
    if result!=[]:
        y=result[0]
        return(y[8:-2]) 
    else:
        return('0')
for i in df['description']:
    serie.append(extract_series(i))
df['serie']=serie
df['serie']=df['serie'].apply(lambda x: '1' if x!='0' else x)
df['serie']=df['serie'].astype(int)

    # recuperer Awards (si awards donc =1 sinon 0)
Awards=[]
def extract_awards(x): 
    r = re.compile(r"Awards:.*\n") 
    result=r.findall(x)
    if result!=[]:
        y=result[0]
        return(y[8:-1])                
    else:
        return('0')
for i in df['description']:
    Awards.append(extract_awards(i))
df['Awards']=Awards
df['Awards']=df['Awards'].apply(lambda x: '1' if x!='0' else x)
df['Awards']=df['Awards'].astype(int)

    # recuperer Like_Percent
Like_Percent =[]
def extract_percentage(x):
    r = re.compile(r"\n\d+% liked this video game")
    result=r.findall(x)
    if result!=[]:
        y=result[0] 
        return(y[1:3]) 
    else:
        return('0')
for i in df['description']:
    Like_Percent.append(extract_percentage(i))
df['Like_Percent']=Like_Percent
df['Like_Percent']=df['Like_Percent'].str.replace(r'%','').astype(int)
z=[]
for i in df['Like_Percent']:
    z.append(i/100)
df['Like_Percent']=z



'''supprimer les jeux non compatible avec vg_salles'''
df.drop(df[df['same_game']==0].index, inplace=True)



'''nettoyage : '''
    # remplacer les nan de la colonne year par 0 et la convertir en int
df['Year']=df['Year'].replace(np.nan, 0)
df['Year']=df['Year'].astype(int)
    # remplacer les nan de la colonne Publisher par not found
df['Publisher']=df['Publisher'].replace(np.nan, 'not found')
    # convertir les types des colonnes:
df['ratings']=df['ratings'].astype(float)
df['nb_votes']=df['nb_votes'].astype(int)
df['metascore']=df['metascore'].astype(int)
df['nb_critic']=df['nb_critic'].astype(int)


        
'''supprimer les colonnes dont on a plus besoin:'''
df.drop(['top','metascorewrap','userscorewrap','nbvotes','nbratings','description',
        'gamename','same_game','userscorewrap2','metascorewrap2','developper'], axis=1,
        inplace=True)



'''reorganisation df '''
df = df[['Rank', 'Name', 'Genre', 'Year','Publisher','Platform','NA_Sales',
      'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales','ratings','nb_votes', 'metascore'
         ,'nb_critic','serie','Like_Percent']] 


'''renomer les colonnes'''
df = df.rename(columns = {'ratings':'userNote','nb_votes':'userNbVote'
                          ,'metascore':'metaNote','nb_critic':'metaNbVote'})


# quantiles (1 = les jeux mauvais...)
df['metaNoteQ']=pd.cut(df['metaNote'], bins=[0,25,50,75,100], labels=[1,2,3,4])
df['metaNoteQ']=df['metaNoteQ'].fillna(1)
df['metaNoteQ']=df['metaNoteQ'].astype(int)


df.head()

,Rank,Name,Genre,Year,Publisher,Platform,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,userNote,userNbVote,metaNote,metaNbVote,serie,Like_Percent,metaNoteQ
0,1,Wii Sports,Sports,2006,Nintendo,Wii,41.49,29.02,3.77,8.46,82.74,8.1,453,76,51,1,0.96,4
1,2,Super Mario Bros.,Platform,1985,Nintendo,NES,29.08,3.58,6.81,0.77,40.24,8.5,98,84,14,1,0.92,4
2,3,Mario Kart Wii,Racing,2008,Nintendo,Wii,15.85,12.88,3.79,3.31,35.82,8.4,1047,82,73,0,0.96,4
3,4,Wii Sports Resort,Sports,2009,Nintendo,Wii,15.75,11.01,3.28,2.96,33.00,8.2,293,80,73,1,0.97,4
5,6,Tetris,Puzzle,1989,Nintendo,GB,23.20,2.26,4.22,0.58,30.26,8.2,258,83,27,0,0.00,4
